## Supervised ML on Descartes Labs Platform: Training a Random Forest Classifier
__________________
This example will demonstrate a typical pattern of generating training data for a supervised classifier using Descartes Labs Platform APIs.

The general steps covered in this notebook are:
* Read in a training dataset from [`Vector`](https://docs.descarteslabs.com/api/vector.html) containing simple land cover categories over the Austin, TX area 
* Visualize our study area and input layers in [`Dynamic Compute`](https://docs.descarteslabs.com/api/dynamic-compute.html)
* Split the area into [`DLTile`s](https://docs.descarteslabs.com/descarteslabs/geo/readme.html#descarteslabs.geo.DLTile)
* Explore feature masking methodologies
* Define an asynchronous [`Function`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function) which takes a tile key as an input and:
    * Searches [`Catalog`](https://docs.descarteslabs.com/descarteslabs/catalog/readme.html) to raster data over the **nir**, **red**, and **green** bands of [National Agricultural Imagery Program (NAIP)](https://app.descarteslabs.com/explorer/datasets/usda:naip:v1) imagery
    * Extracts intersecting features as raster masks
    * Returns associated pixel values as lists
    
Move on to [02b Training a Supervised Classifier.ipynb](02b%20Training%20a%20Supervised%20Classifier.ipynb) to retrieve the results of the completed function and train a simple Random Forest Classifier. 

In [ ]:
import json
import os
import pickle
import yaml
import sys

import descarteslabs as dl
import descarteslabs.compute
import descarteslabs.vector as dl_vector
import descarteslabs.dynamic_compute as dc
import geopandas as gpd
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio.mask
import shapely.geometry as sgeom

Load global variables for reference throughout this example, including the NAIP product ID, a list of bands, a start and end date, resolution, and a name for our function:

In [ ]:
with open("config.yaml", "r") as file:
    config = yaml.load(file, yaml.FullLoader)

In [ ]:
major = sys.version_info.major
minor = sys.version_info.minor
compute_image = f"python{major}.{minor}:latest"
compute_image

Next we retrieve a table of sample training features:

In [ ]:
table = dl_vector.Table.get(config["training_table_name"])
table

## Study Area - Austin, TX
In the next few cells we will set up an interactive map frame to overlay our training feature collection on the input NAIP imagery. 

Setting up an interactive map, alongside center coordinates and zoom:

In [ ]:
m = dc.map
m.center = 30.2552, -97.7689
m.zoom = 12

Create a mosaic of our NAIP imagery and visualize as a false color composite (FCC):

In [ ]:
naip_mosaic = dc.Mosaic.from_product_bands(
    config["product_id"],
    config["bands"],
    start_datetime=config["start"],
    end_datetime=config["end"],
)
naip_mosaic.visualize("FCC", m)

Next visualize our input training table:

In [ ]:
table.visualize(
    "Training Polygons",
    m,
)

### Generating Training Data - Tiling
As outlined above, the general steps to extract training data are as follows:
* Splitting up the training AOI into tiles
* For each tile we search NAIP imagery and extract all intersecting feature masks

First, lets pull the data from the table and get a feel for it.

In [ ]:
gdf = table.collect()
gdf.head()

We have rows of data with a geometry column, a plain-text category, a category integer (so water maps to the value 3) and a uuid that uniquely identifies each row. Lets look at what the categories are

In [ ]:
print(
    f"There are {len(gdf)} features with the following categories: {gdf.category.unique()}"
)

Now we will split our input feature collection into tiles over which we will define our function to iterate.

In [ ]:
gdf_geom = sgeom.box(*gdf["geometry"].total_bounds)
dltiles = dl.geo.DLTile.from_shape(
    gdf_geom, resolution=config["resolution_m"], tilesize=2048, pad=0
)
len(dltiles)

Since our feature collection is sparse and NAIP is high resolution, we want to omit any tiles that don't intersect any of our input features:

In [ ]:
dltiles = [dltile for dltile in dltiles if gdf.intersects(dltile.geometry).any()]
len(dltiles)

Lastly, we can add our tile geometries to the map to visualize our project:

In [ ]:
dltile_gdf = gpd.GeoDataFrame(
    {
        "geometry": [dltile.geometry for dltile in dltiles],
    },
    crs=4326,
)
geo_data = ipyleaflet.GeoData(
    geo_dataframe=dltile_gdf,
    style={"color": "grey", "fillOpacity": 0.0},
    name="DLTiles",
)
m.add_layer(geo_data)

In [ ]:
m

## Generating Training Data - Masking

Next up we will explore feature masking methodologies. In this example we will retrieve imagery over a sample tile as a geotiff, mask the intersecting features, and return the corresponding band values. 

First we will search NAIP over a sample tile:

In [ ]:
dltile = dltiles[0]

In [ ]:
naip_ic = (
    dl.catalog.Product.get(config["product_id"])
    .images()
    .intersects(dltile)
    .filter(config["start"] <= dl.catalog.properties.acquired < config["end"])
    .sort("acquired")
    .limit(None)
).collect()
naip_ic

Next we define a function called `generate_polygon_masks`. In this function we will use [`rasterio.mask`](https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html) to efficienty mask our geotiff downloaded through Catalog to each feature in our training dataset and extract the associated band values into our geodataframe. 

The function takes 3 arguments:
* An input row in a geodataframe
* An opened raster dataset
* A list of bands for column names

It then passes the row's geometry as a feature mask, including **all_touched** to find all pixels that touch our geometry and **crop** to efficiently open a small _window_ of the whole raster. Iterating over each band name, it finally returns a list of corresponding pixel values for each feature (e.g. **nir**, **red**, and **green**).

In [ ]:
def generate_polygon_masks(
    row: pd.Series, in_dataset: rasterio.io.DatasetWriter, bands: list[str]
) -> pd.Series:
    """Injects for each band the masked pixels for the input geometry in row"""
    # Perform the mask--this returns a feature mask, transform (unused),
    # and the window over which to read the input dataset
    mask, transform, window = rasterio.mask.raster_geometry_mask(
        in_dataset, [row["geometry"]], all_touched=True, crop=True
    )
    # Opens the input dataset at the specified window
    window_arr = in_dataset.read(window=window)
    # For each band we mask to the feature and return the stack
    # Arr is shape (bands, y, x)
    out_arr = np.stack([arr[~mask] for arr in window_arr])
    # Figuring out out to best store this--we just return a list of values
    # for each band in the input dataset
    for idx, band in enumerate(bands):
        vals = out_arr[idx].tolist()
        row[f"{band}"] = vals

    return row

Here we will test things out and plot the steps below:
1. Download our NAIP imagery as a geotiff from Catalog
2. Open our geotiff in rasterio
4. Reproject our geodataframe to local CRS
3. Apply our `generate_polygon_masks` function to annotate each feature with contained band values

We also will plot out the corresponding windowed masks for the **crop** argument explained above. 

In [ ]:
print("Downloading mosaic...")
naip_ic.download_mosaic(config["bands"], dest="naip_temp.tif")
# Performing the feature sampling by applying the function defined above:
with rasterio.open("naip_temp.tif", "r+") as in_dataset:
    print("Performing feature sampling...")
    # Generate Polygon Masks function:
    table = dl_vector.Table.get(config["training_table_name"], aoi=dltile)
    sampled_gdf = table.collect().to_crs(dltile.crs)
    sampled_gdf = sampled_gdf.apply(
        lambda row: generate_polygon_masks(row, in_dataset, config["bands"]), axis=1
    )
    # Visualizing methodology:
    # 1. Cropped feature mask:
    out_crp_msk, out_crp_trans, out_crp_window = rasterio.mask.raster_geometry_mask(
        in_dataset, sampled_gdf.geometry.tolist(), all_touched=True, crop=True
    )
    # 2. Uncropped feature mask:
    out_msk, out_trans, out_window = rasterio.mask.raster_geometry_mask(
        in_dataset, sampled_gdf.geometry.tolist(), all_touched=True, crop=False
    )

    fig, ax = plt.subplots(figsize=(9, 3), nrows=1, ncols=3)
    arr = in_dataset.read()
    ax[0].imshow(arr.transpose((1, 2, 0))[:, :, :3])
    ax[0].set_title(r"FCC")
    ax[1].imshow(out_msk)
    ax[1].set_title(r"Uncropped Mask")
    ax[2].imshow(out_crp_msk)
    ax[2].set_title(r"Cropped Mask")
plt.tight_layout()
os.remove("naip_temp.tif")

And here we see the new columns on our dataframe:

In [ ]:
sampled_gdf.head()

## Putting it All Together with Batch Compute
Here we'll define a function which wraps all of the previously outlined methodology into a self-contained Python function. The inputs here are a single tile key and the overall steps are as follows:
* Re-create a tile from the passed key
* Retrieve the training features clipped to the input tile
* Search NAIP over the input tile and retrieve the imagery as a geotiff
* Perform the feature sampling method outlined above against the clipped features
* Return the associated intersecting band values as lists

In [ ]:
def get_pixel_values(dltile_key: str):

    import json
    import os

    import descarteslabs as dl
    import descarteslabs.vector as dl_vector
    import geopandas as gpd
    import numpy as np
    import pandas as pd
    import rasterio
    import rasterio.mask

    def generate_polygon_masks(
        row: pd.Series, in_dataset: rasterio.io.DatasetWriter, bands: list[str]
    ) -> pd.Series:
        """Injects for each band the masked pixels for the input geometry in row"""
        mask, transform, window = rasterio.mask.raster_geometry_mask(
            in_dataset, [row["geometry"]], all_touched=True, crop=True
        )
        window_arr = in_dataset.read(window=window)
        out_arr = np.stack([arr[~mask] for arr in window_arr])
        for idx, band in enumerate(bands):
            vals = out_arr[idx].tolist()
            row[f"{band}"] = vals

        return row

    dltile = dl.geo.DLTile.from_key(dltile_key)
    print(f"Processing {dltile_key}")

    TABLE_ID = "descarteslabs:austin-landcover-training-data"
    PRODUCT_ID = "usda:naip:v1"
    START = "2020-01-01"
    END = "2021-01-01"
    BANDS = ["nir", "red", "green"]
    # Pulling GDF from Vector

    table = dl_vector.Table.get(TABLE_ID, aoi=dltile)
    gdf = table.collect().to_crs(dltile.crs)

    if len(gdf) == 0:
        return {}

    print("Searching Images...")
    naip_ic = (
        dl.catalog.Product.get(PRODUCT_ID)
        .images()
        .intersects(dltile)
        .filter(START <= dl.catalog.properties.acquired < END)
        .sort("acquired")
        .limit(None)
    ).collect()
    print(naip_ic)

    print("Downloaded GDF...")
    naip_ic.download_mosaic(
        bands=BANDS,
        geocontext=dltile,
        dest=f"naip_temp.tif",
        format="tif",
    )
    print("Downloaded GeoTIFF...")

    # Opening the geotiff via Rasterio
    with rasterio.open(f"naip_temp.tif", "r+") as in_ds:
        print("Performing feature sampling...")
        # Generate Polygon Masks function:
        sampled_gdf = gdf.apply(
            lambda row: generate_polygon_masks(row, in_ds, BANDS), axis=1
        )

    # Returning GDF as a dictionary, dropping geom column along the way:
    out_data = sampled_gdf.drop(columns=["geometry"]).to_dict()

    print("Cleaning up")
    # Deleting the tiff from memory
    os.remove(f"naip_temp.tif")
    print("Complete")

    return {"dltile": dltile_key, "data": out_data}

Now that it's all packaged up into a function, we can test it locally:

In [ ]:
pd.DataFrame(get_pixel_values(dltiles[0].key)["data"])

Now we format our input arguments:

In [ ]:
args = [[dltile.key] for dltile in dltiles]
len(args)

Once we are happy with the performance of our function we can save it to our Compute service.

Note here that we must pass geopandas as a requirement:

In [ ]:
async_func = dl.compute.Function(
    get_pixel_values,
    name=config["gen_data_func_name"],
    image=compute_image,
    cpus=1,
    memory=2,
    timeout=300,
    maximum_concurrency=20,
    retry_count=1,
)

async_func.save()
print(f"Saved {async_func.id}")

**_Take note of your Function ID!_**

And finally map args to our Function to return a set of jobs:

In [ ]:
jobs = async_func.map(args)

Navigate to [app.descarteslabs.com/compute](https://app.descarteslabs.com/compute) to track your progress.

Or wait programmatically via:

In [ ]:
# async_func.wait_for_completion()

Once this function completes, you can move on to [02b Training a Supervised Classifier.ipynb](02b%20Training%20a%20Supervised%20Classifier.ipynb) to retrieve the results and train our model! 